# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import time

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [9]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

639

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [11]:
url = "http://api.openweathermap.org/data/2.5/weather?"

record_count = 1
set_count = 1

print("Beginning Data Retrieval")
print("------------------------")
for i, city in enumerate(cities):
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    query_url = url + "q=" + city + "&APPID=" + weather_api_key
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    record_count += 1
    time.sleep(2)

Beginning Data Retrieval
------------------------
Processing Record 1 of Set 1 | kirakira
Processing Record 2 of Set 1 | port alfred
Processing Record 3 of Set 1 | rikitea
Processing Record 4 of Set 1 | katsuura
Processing Record 5 of Set 1 | hithadhoo
Processing Record 6 of Set 1 | tazovskiy
Processing Record 7 of Set 1 | necochea
Processing Record 8 of Set 1 | thompson
Processing Record 9 of Set 1 | kavieng
Processing Record 10 of Set 1 | albany
Processing Record 11 of Set 1 | nanortalik
Processing Record 12 of Set 1 | avarua
Processing Record 13 of Set 1 | brae
Processing Record 14 of Set 1 | veraval
Processing Record 15 of Set 1 | taolanaro
Processing Record 16 of Set 1 | afonso bezerra
Processing Record 17 of Set 1 | tommot
Processing Record 18 of Set 1 | kaitangata
Processing Record 19 of Set 1 | arraial do cabo
Processing Record 20 of Set 1 | poronaysk
Processing Record 21 of Set 1 | mataura
Processing Record 22 of Set 1 | half moon bay
Processing Record 23 of Set 1 | sorong
Pro

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
name = []
lat = []
lng = []
max_temp = []
humidity = []
cloud = []
wind = []
country = []
date = []

for city in cities:
    weather_response = requests.get(query_url).json()
    
    name.append(weather_response['name'])
    lat.append(weather_response['coord']['lat'])
    lng.append(weather_response['coord']['lon'])
    max_temp.append(weather_response['main']['temp_max'])
    humidity.append(weather_response['main']['humidity'])
    cloud.append(weather_response['clouds']['all'])
    wind.append(weather_response['wind']['speed'])
    country.append(weather_response['sys']['country'])
    date.append(weather_response['dt'])
    
    time.sleep(2)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [4]:
#  Get the indices of cities that have humidity over 100%.


In [5]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression